In [ ]:
# !pip install git+https://github.com/modAL-python/modAL.git

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling

In [2]:
df = pd.read_csv('hw1_result_df.csv')

In [4]:
import re

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'\s+', ' ', text)
        text = text.strip().lower()
    else:
        text = ""
    return text

df['document_ru_clean'] = df['document_ru'].apply(clean_text)
df['question_ru_clean'] = df['question_ru'].apply(clean_text)
df['answer_ru_clean'] = df['answer_ru'].apply(clean_text)

df['instruct'] = " Тебе дан документ со следующим содержанием: \n" + df['document_ru_clean'] + " \n\nОтветь на вопрос: " + df['question_ru_clean']

In [6]:
import numpy as np

X = df['instruct'].values
y = df['answer_ru'].values

vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

X_train_indices = np.arange(X_vectorized.shape[0])
X_train_data, X_test_data, y_train_data, y_test_data, train_indices, test_indices = train_test_split(
    X_vectorized, y, X_train_indices, test_size=0.2, random_state=42
)

Выберем небольшое подмножество для изначальной разметки

In [7]:
n_initial = int(X_train_data.shape[0] * 0.1)
initial_idx = np.random.choice(range(X_train_data.shape[0]), size=n_initial, replace=False)
X_initial = X_train_data[initial_idx]
y_initial = np.array([y_train_data[i] for i in initial_idx])

pool_idx = np.setdiff1d(range(X_train_data.shape[0]), initial_idx)
X_pool = X_train_data[pool_idx]
y_pool = np.array([y_train_data[i] for i in pool_idx])

In [8]:
original_indices = train_indices[pool_idx] # оригинальные индексы для отслеживания выбранных примеров

Создаем и обучаем активного ученика

In [9]:
learner = ActiveLearner(
    estimator=MultinomialNB(),
    X_training=X_initial, 
    y_training=y_initial,
    query_strategy=uncertainty_sampling
)

В данном примере используется стратегия выбора наибольшей неопределенности - она выбирает те примеры из пула, предсказания по которым наиболее "неуверенные"

In [11]:
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

selected_indices = []

n_queries = 100 
for i in tqdm(range(n_queries)):
    query_idx, query_instance = learner.query(X_pool, n_instances=1) # выбираем пример, в котором модель наименее уверена
    
    original_idx = original_indices[query_idx[0]]
    selected_indices.append(original_idx)
    
    X_query = X_pool[query_idx]
    y_query = np.array([y_pool[i] for i in query_idx])
    
    learner.teach(X_query, y_query) # обучаем модель с новым примером
    
    # удаляем размеченный пример из пула
    mask = np.ones(X_pool.shape[0], dtype=bool)
    mask[query_idx] = False
    X_pool = X_pool[mask]
    y_pool = np.array([y for i, y in enumerate(y_pool) if i not in query_idx])
    original_indices = original_indices[mask]
    

  0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
print("Примеры для разметки:")
for idx in selected_indices[:10]: # запринтим только первые 10
    question = df.iloc[idx]['instruct']
    answer = df.iloc[idx]['answer_ru']
    print(f"\nВопрос: {question}")
    print(f"\nОтвет: {answer}")
    print("-" * 100)
    
selected_samples = df.iloc[selected_indices]
selected_samples.to_csv('hw4_selected_samples_for_annotation.csv', index=False)

Примеры для разметки:

Вопрос:  Тебе дан документ со следующим содержанием: 
около 20-го века все большее население азианцев жило в санта-монике и вене. японская рыбацкая деревня была расположена рядом с длинным причалом, в то время как небольшое число китайцев проживало или работало как в санта-муниках, так и в венисе. оба этнических меньшинства часто рассматривались по-разному белыми американцами, которые часто были хорошо привязаны к японцам, но снисходительны к китайцам. японский деревенский рыбак был неотъемлемой экономической частью общины санта-моника. 

Ответь на вопрос: какую роль сыграла рыбацкая деревня в санта-монике?

Ответ: экономическая часть
----------------------------------------------------------------------------------------------------

Вопрос:  Тебе дан документ со следующим содержанием: 
однако в стандартном подходе учитываются только асимптотические свойства таких функций, поскольку они не зависят от эталонного компьютера. мы утверждаем, что этот подход может бы

### Подход к стратегии выбора данных для доразметки:
Использовала стратегию uncertainty_sampling. Эта стратегия выбирает те примеры из пула, для которых текущая модель делает наименее уверенные предсказания -> так можно обучить модель наиболее информативным примерам, улучшая общую способность модели на предсказания на новых данных.

### Отчет о данных
В этом процессе в selected_samples_for_annotation.csv сохраняются данные, которые были выбраны моделью для доразметки. 

В перспективе работы с этими данными планирую использовать более сложные модели, дособрать еще датасет на закрытых документах и использовать дополнительные функции для предобработки данных.